In [66]:
# todo
# 1. read train_edges from disk
# 2. create graph
# 3. setup params
# 4. generate temporal walks
# 5. train word2vec

In [67]:
import random
import pickle
import networkx as nx

seed = 42
random.seed(seed)

In [68]:
train_edges_file = '../datasets/collegemsg/train_edges.pkl'

with open(train_edges_file, 'rb') as te:
    train_edges = pickle.load(te)

In [69]:
graph = nx.MultiDiGraph()
graph.add_edges_from(train_edges)

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 2,
 1,
 0,
 3,
 3,
 4,
 0,
 1,
 2,
 0,
 5,
 3,
 4,
 6,
 7,
 1,
 8,
 9,
 10,
 5,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 11,
 12,
 0,
 1,
 0,
 2,
 0,
 1,
 6,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 7,
 3,
 2,
 4,
 3,
 4,
 5,
 0,
 1,
 0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 2,
 11,
 0,
 12,
 6,
 7,
 8,
 13,
 14,
 15,
 0,
 16,
 17,
 18,
 19,
 20,
 9,
 0,
 0,
 10,
 1,
 2,
 3,
 11,
 12,
 13,
 14,
 15,
 16,
 2,
 21,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 17,
 18,
 19,
 20,
 22,
 21,
 1,
 8,
 22,
 23,
 23,
 24,
 24,
 0,
 25,
 1,
 26,
 27,
 25,
 28,
 26,
 9,
 11,
 29,
 27,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 3,
 4,
 5,
 2,
 6,
 3,
 0,
 0,
 0,
 0,
 1,
 4,
 0,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 1,
 4,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 5,
 6,
 0,
 1,
 0,
 2,
 3,
 0,
 0,
 4,
 7,
 0,
 

In [104]:
R = 80 
N = len(graph.nodes)
omega = 10 # context window size
L = 80 # max length of walk
beta = R * N * (L - omega + 1)
beta = 1000

In [105]:
C = 0
temporal_walks = []
edges = list(graph.edges(data=True))

In [106]:
while beta - C > 0:
    walk = []
    first_edge = random.sample(edges, k=1) # unbiased first edge 
    u, v = first_edge[0][:2]
    walk.append(u)
    walk.append(v)
    vtime = first_edge[0][2]['timestamp']
    
    for i in range(L - 2):
        vneighbors = list(graph.out_edges(v, data=True))
        temp_neighbors = list(filter(lambda x: x[2]['timestamp'] >= vtime, vneighbors))
        if temp_neighbors:
            next_edge = random.sample(temp_neighbors, k=1)[0]
            v, vtime = next_edge[1], next_edge[2]['timestamp']
            walk.append(v)
            
    if len(walk) >= omega:
        temporal_walks.append(walk)
        C += len(walk) - omega + 1

In [108]:
temporal_walks

[[220, 214, 302, 557, 605, 479, 1033, 576, 1339, 323],
 [189, 712, 1042, 1260, 32, 1183, 32, 8, 1238, 1272],
 [3, 221, 368, 542, 783, 181, 783, 1004, 908, 1354, 128, 502, 540],
 [605, 413, 683, 646, 638, 996, 638, 758, 681, 212],
 [652, 598, 878, 1072, 1158, 1423, 1158, 1072, 1158, 1072],
 [952, 830, 454, 1043, 454, 1043, 454, 1043, 495, 649],
 [365, 540, 495, 498, 378, 254, 1039, 254, 1260, 254, 605],
 [36, 189, 712, 1037, 447, 249, 447, 249, 1644, 995, 1644, 1686, 710],
 [103, 462, 913, 41, 975, 939, 831, 1187, 831, 1049],
 [194, 62, 475, 75, 475, 250, 753, 984, 318, 67],
 [263, 316, 355, 316, 355, 316, 325, 874, 919, 103],
 [681, 758, 762, 713, 950, 1346, 1472, 1346, 518, 1339],
 [323, 834, 758, 741, 323, 741, 1280, 725, 835, 725, 835, 725, 323],
 [323, 341, 1255, 1332, 1781, 1783, 1675, 1783, 1624, 95],
 [1056, 687, 1056, 1042, 1260, 32, 1577, 32, 518, 249],
 [321, 392, 739, 392, 353, 950, 353, 1183, 1354, 128, 502, 398, 1836, 1724],
 [9, 527, 74, 73, 617, 103, 638, 950, 353, 950, 